In [21]:
#data handling
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MinMaxScaler

#feature selection
from sklearn.feature_selection import mutual_info_classif

#classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# performance metrics
from sklearn.metrics import balanced_accuracy_score,f1_score,precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

In [22]:
df=pd.read_csv('mar_30.csv',sep=';',index_col=0)

In [23]:
df.shape

(804, 61)

In [24]:
#Data Preprocessing:

X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [50]:
#Encoding the labels:

label_encoder=LabelEncoder()
label_encoder.fit(y)
y=label_encoder.transform(y)
labels=label_encoder.classes_
classes=np.unique(y)


In [51]:
labels

array(['normal', 'tumoral'], dtype=object)

In [52]:
classes

array([0, 1])

In [53]:
#Split the data:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [42]:
#1. Mean

#Imputation of the data:


from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
X_train_imputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)
X_test_imputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [ ]:
#2.Median

#Imputation of the data:


from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='median')
X_train_imputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)
X_test_imputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [ ]:
#3.Constant

#Imputation of the data:


from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='constant',fill_values=0)
X_train_imputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)
X_test_imputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [ ]:
#4.KNN imputer

from sklearn.impute import KNNImputer

# For training set
knn_imputer_train = KNNImputer(n_neighbors=5)
X_train_imputed= pd.DataFrame(knn_imputer_train.fit_transform(X_train), columns=X_train.columns)

# For testing set
knn_imputer_test = KNNImputer(n_neighbors=5)
X_test_imputed= pd.DataFrame(knn_imputer_test.fit_transform(X_test), columns=X_test.columns)


In [ ]:
#5.Iterative imputation
#iterative imputation:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Initialize the IterativeImputer
imputer = IterativeImputer(random_state=42)

# Fit the imputer on the training data and transform both training and testing data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert the imputed arrays back to DataFrames
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns, index=X_test.index)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import multivariate_normal as mvn
from scipy.stats import gamma

class BPCA(object):

    def __init__(self, a_alpha=1e-3, b_alpha=1e-3, a_tau=1e-3, b_tau=1e-3, beta=1e-3):
        # hyperparameters
        self.a_alpha = a_alpha # parameter of alpha's prior (a Gamma distribution)
        self.b_alpha = b_alpha # parameter of alpha's prior (a Gamma distribution)
        self.a_tau = a_tau     # parameter of tau's prior (a Gamma distribution)
        self.b_tau = b_tau     # parameter of tau's prior (a Gamma distribution)
        self.beta = beta
        # history of ELBOS
        self.elbos = None
        self.variations = None
        # history of log likelihoods
        self.loglikelihoods = None


    def update(self):
        """fixed-point update of the Bayesian PCA"""
        # inverse of the sigma^2
        self.tau = self.a_tau_tilde / self.b_tau_tilde
        # hyperparameters controlling the magnitudes of each column of the weight matrix
        self.alpha = self.a_alpha_tilde / self.b_alpha_tilde
        # covariance matrix of the latent variables
        self.cov_z = np.linalg.inv(np.eye(self.q) + self.tau *
                        (np.trace(self.cov_w) + np.dot(self.mean_w.T, self.mean_w)))
        # mean of the latent variable
        self.mean_z = self.tau * np.dot(np.dot(self.cov_z, self.mean_w.T), self.Xb - self.mean_mu)
        # covariance matrix of the mean observation
        self.cov_mu = np.eye(self.d) / (self.beta + self.b * self.tau)
        # mean of the mean observation
        self.mean_mu = self.tau * np.dot(self.cov_mu, np.sum(self.Xb-np.dot(self.mean_w,
                        self.mean_z), axis=1)).reshape(self.d, 1)
        # covariance matrix of each column of the weight matrix
        self.cov_w = np.linalg.inv(np.diag(self.alpha) + self.tau *
                        (self.b * self.cov_z + np.dot(self.mean_z, self.mean_z.T)))
        # mean of each column of the weight matrix
        self.mean_w = self.tau * np.dot(self.cov_w, np.dot(self.mean_z, (self.Xb-self.mean_mu).T)).T
        # estimation of the b in alpha's Gamma distribution
        self.b_alpha_tilde = self.b_alpha + 0.5 * (np.trace(self.cov_w) +
                        np.diag(np.dot(self.mean_w.T, self.mean_w)))
        # estimation of the b in tau's Gamma distribution
        self.b_tau_tilde = self.b_tau + 0.5 * np.trace(np.dot(self.Xb.T, self.Xb)) + \
                        0.5 * self.b*(np.trace(self.cov_mu)+np.dot(self.mean_mu.flatten(), self.mean_mu.flatten())) + \
                        0.5 * np.trace(np.dot(np.trace(self.cov_w)+np.dot(self.mean_w.T, self.mean_w),
                                        self.b*self.cov_z+np.dot(self.mean_z, self.mean_z.T))) + \
                        np.sum(np.dot(np.dot(self.mean_mu.flatten(), self.mean_w), self.mean_z)) + \
                        -np.trace(np.dot(self.Xb.T, np.dot(self.mean_w, self.mean_z))) + \
                        -np.sum(np.dot(self.Xb.T, self.mean_mu))


    def calculate_log_likelihood(self):
        """calculate the log likelihood of observing self.X"""
        w = self.mean_w
        c = np.eye(self.d)*self.tau + np.dot(w, w.T)
        xc = self.X - self.X.mean(axis=1).reshape(-1,1)
        s = np.dot(xc, xc.T) / self.N
        self.s = s
        c_inv_s = scipy.linalg.lstsq(c, s)[0]
        loglikelihood = -0.5*self.N*(self.d*np.log(2*np.pi)+np.log(np.linalg.det(c))+np.trace(c_inv_s))
        return loglikelihood


    def calculate_ELBO(self):
        '''ELBO = E_q[-log(q(theta))+log(p(theta)+log(p(Y|theta,X)))]
                = -entropy + logprior + loglikelihood '''

        # random sample
        z = np.array([np.random.multivariate_normal(self.mean_z[:,i], self.cov_z) for i in range(self.b)]).T
        mu = np.random.multivariate_normal(self.mean_mu.flatten(), self.cov_mu)
        w = np.array([np.random.multivariate_normal(self.mean_w[i], self.cov_w) for i in range(self.d)])
        alpha = np.random.gamma(self.a_alpha_tilde, 1/self.b_alpha_tilde)
        tau = np.random.gamma(self.a_tau_tilde, 1/self.b_tau_tilde)

        # entropy
        # q(z)
        entropy = np.sum(np.array([mvn.logpdf(z[:,i], self.mean_z[:,i], self.cov_z) for i in range(self.b)]))

        # q(mu)
        entropy += mvn.logpdf(mu, self.mean_mu.flatten(), self.cov_mu)

        # q(W)
        entropy += np.sum(np.array([mvn.logpdf(w[i], self.mean_w[i], self.cov_w) for i in range(self.d)]))

        # q(alpha)
        entropy += np.sum(gamma.logpdf(alpha, self.a_alpha_tilde, scale=1/self.b_alpha_tilde))

        # q(tau)
        entropy += gamma.logpdf(tau, self.a_tau_tilde, scale=1/self.b_tau_tilde)

        # logprior
        # p(z), z ~ N(0, I)
        logprior = np.sum(np.array([mvn.logpdf(z[:,i], mean=np.zeros(self.q), cov=np.eye(self.q)) for i in range(self.b)]))

        # p(w|alpha), conditional gaussian
        logprior += np.sum(np.array([self.d/2*np.log(alpha[i]/(2*np.pi))-alpha[i]*np.sum(w[:,i]**2)/2 for i in range(self.q)]))

        # p(alpha), alpha[i] ~ Gamma(a, b)
        logprior += np.sum(gamma.logpdf(alpha, self.a_alpha, scale=1/self.b_alpha))

        # p(mu), mu ~ N(0, I/beta)
        logprior += mvn.logpdf(mu, mean=np.zeros(self.d), cov=np.eye(self.d)/self.beta)

        # p(tau), tau ~ Gamma(c, d)
        logprior += gamma.logpdf(tau, self.a_tau, scale=1/self.b_tau)

        # loglikelihood
        pred = np.dot(w, z) + mu.reshape(-1,1)
        loglikelihood = np.sum(np.array([mvn.logpdf(self.Xb[:,i], pred[:,i], np.eye(self.d)/tau) for i in range(self.b)]))

        return -entropy + logprior + loglikelihood


    def batch_idx(self, i):
        if self.b == self.N:
            return np.arange(self.N)
        idx1 = (i*self.b) % self.N
        idx2 = ((i+1)*self.b) % self.N
        if idx2 < idx1:
            idx1 -= self.N
        return np.arange(idx1, idx2)


    def fit(self, X=None, batch_size=128, iters=500, print_every=100, verbose=False, trace_elbo=False, trace_loglikelihood=False):
        """fit the Bayesian PCA model using fixed-point update"""
         # data, # of samples, dims
        self.X = X.T # don't need to transpose X when passing it
        self.d = self.X.shape[0]
        self.N = self.X.shape[1]
        self.q = self.d-1
        self.ed = []
        self.b = min(batch_size, self.N)

        # variational parameters
        self.mean_z = np.random.randn(self.q, self.b) # latent variable
        self.cov_z = np.eye(self.q)
        self.mean_mu = np.random.randn(self.d, 1)
        self.cov_mu = np.eye(self.d)
        self.mean_w = np.random.randn(self.d, self.q)
        self.cov_w = np.eye(self.q)
        self.a_alpha_tilde = self.a_alpha + self.d/2
        self.b_alpha_tilde = np.abs(np.random.randn(self.q))
        self.a_tau_tilde = self.a_tau + self.b * self.d / 2
        self.b_tau_tilde = np.abs(np.random.randn(1))

        # update
        order = np.arange(self.N)
        elbos = np.zeros(iters)
        loglikelihoods = np.zeros(iters)
        for i in range(iters):
            idx = order[self.batch_idx(i)]
            self.Xb = self.X[:,idx]
            self.update()
            if trace_elbo:
                elbos[i] = self.calculate_ELBO()
            if trace_loglikelihood:
                loglikelihoods[i] = self.calculate_log_likelihood()
            if verbose and i % print_every == 0:
                print('Iter %d, LL: %f, alpha: %s' % (i, loglikelihoods[i], str(self.alpha)))
        self.captured_dims()
        self.elbos = elbos if trace_elbo else None
        self.loglikelihoods = loglikelihoods if trace_loglikelihood else None


    def captured_dims(self):
        """return the number of captured dimensions"""
        sum_alpha = np.sum(1/self.alpha)
        self.ed = np.array([i for i, inv_alpha in enumerate(1/self.alpha) if inv_alpha < sum_alpha/self.q])


    def transform(self, X=None, full=True):
        """generate samples from the fitted model"""
        X = self.X if X is None else X.T
        if full:
            w = self.mean_w
            l = self.q
        else:
            w = self.mean_w[:,ed]
            l = len(self.ed)
        m = np.eye(l)*self.tau + np.dot(w.T, w)
        inv_m = np.linalg.inv(m)
        z = np.dot(np.dot(inv_m, w.T), X - self.mean_mu)
        return z.T
        # return np.array([np.random.multivariate_normal(z[:,i], inv_m*self.tau) for i in range(X.shape[1])])


    def inverse_transform(self, z, full=True):
        """transform the latent variable into observations"""
        z = z.T
        if full:
            w = self.mean_w
        else:
            w = self.mean_w[:,ed]
        x = np.dot(w, z) + self.mean_mu
        return x.T
        # return np.array([np.random.multivariate_normal(x[:,i], np.eye(self.d)*self.tau) for i in range(z.shape[1])])


    def fit_transform(self, X=None, batch_size=128, iters=500, print_every=100, verbose=False, trace_elbo=False, trace_loglikelihood=False):
        self.fit(X, batch_size, iters, print_every, verbose, trace_elbo)
        return self.transform()


    def generate(self, size=1):
        """generate samples from the fitted model"""
        w = self.mean_w[:, self.ed]
        c = np.eye(self.d)*self.tau + np.dot(w, w.T)
        return np.array([np.random.multivariate_normal(self.mean_mu.flatten(), c) for i in range(size)])


    def get_weight_matrix(self):
        return self.mean_w


    def get_inv_variance(self):
        return self.alpha


    def get_effective_dims(self):
        return len(self.ed)


    def get_cov_mat(self):
        w = self.mean_w[:, self.ed]
        c = np.eye(self.d)*self.tau + np.dot(w, w.T)
        return c


    def get_elbo(self):
        return self.elbos


    def get_loglikelihood(self):
        return self.loglikelihoods

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler



# Function to perform Bayesian PCA Imputation
def bayesian_pca_impute(data, n_components=None, max_iter=100, tol=1e-4):
    # Step 1: Initial Imputation using mean imputation
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    data_imputed = imp_mean.fit_transform(data)

    # Step 2: Standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_imputed)

    # Step 3: Define Bayesian PCA model
    bpca = BPCA()

    # Step 4: Fit the BPCA model
    bpca.fit(data_scaled.T, batch_size=32, iters=max_iter, verbose=True, trace_elbo=True, trace_loglikelihood=True)

    # Step 5: Transform and inverse transform to impute missing values
    data_imputed_scaled = bpca.inverse_transform(bpca.transform(data_scaled.T)).T

    # Step 6: Reverse standardization
    data_final = scaler.inverse_transform(data_imputed_scaled)

    return pd.DataFrame(data_final, columns=data.columns)



# Perform Bayesian PCA Imputation
X_train_imputed = bayesian_pca_impute(X_train, n_components=2)
X_test_imputed = bayesian_pca_impute(X_test, n_components=2)




/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: overflow encountered in det
  


Iter 0, LL: -inf, alpha: [8.14632232e+02 2.24384010e+02 3.08143878e+02 2.41346572e+02
 2.74982784e+02 6.01395258e+02 5.53117567e+02 2.24044378e+02
 2.94894641e+03 7.49280972e+02 3.19545132e+02 9.49943404e+02
 2.88537742e+02 1.84876884e+02 2.23654376e+02 3.70270411e+02
 4.39018815e+02 4.47034079e+03 1.78635921e+02 6.80004271e+02
 2.15423264e+03 2.96820255e+02 2.83811601e+02 2.21144949e+02
 4.16907522e+02 1.87356974e+02 2.35355398e+03 2.19929178e+02
 1.58728098e+04 4.66507022e+02 1.84682304e+02 2.01299318e+03
 3.23760375e+02 2.33720923e+02 1.79952199e+02 8.80003006e+03
 3.29841828e+02 1.48468631e+03 3.77150339e+02 1.38151700e+03
 4.79935003e+02 3.71764166e+02 3.65704126e+02 1.63811058e+04
 1.73729745e+02 4.41720227e+02 4.39283112e+02 3.25769448e+02
 4.49396767e+02 2.41967604e+02 7.11032104e+02 2.39309138e+03
 2.28989313e+02 8.14151822e+02 5.43503019e+03 6.27040565e+02
 7.74834605e+04 8.82798547e+02 3.87615879e+02 5.32535739e+02
 6.45017276e+03 7.19926121e+02 1.18569587e+05 4.55920956e+02

/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: overflow encountered in det
  


In [54]:
# beta-vae

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Assuming X_train and X_test are your dataframes with missing values
# Assuming y_train and y_test are your target labels for classification

# Impute missing values with a simple strategy (e.g., median)
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Normalize the data
scaler = StandardScaler()
X_train_imputed = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=X_train.columns)
X_test_imputed = pd.DataFrame(scaler.transform(X_test_imputed), columns=X_test.columns)

X_train_imputed = X_train_imputed.astype(np.float32)
X_test_imputed = X_test_imputed.astype(np.float32)

# Ensure labels are integers
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

input_dim = X_train_imputed.shape[1]
latent_dim = int(input_dim * 0.3)
n_classes = len(np.unique(y_train))  # Number of classes for classification

# VAE imputation with classification head
class VAE(tf.keras.Model):
    def __init__(self, input_dim, latent_dim, layers_config, n_classes):
        super(VAE, self).__init__()
        encoder_layers = []
        if len(layers_config) > 0:
            encoder_layers.append(layers.InputLayer(input_shape=(input_dim,)))
            for units in layers_config:
                encoder_layers.append(layers.Dense(units, activation='relu'))
                encoder_layers.append(layers.BatchNormalization())
            encoder_layers.append(layers.Dense(latent_dim * 2))  # mean and log-variance
        else:
            encoder_layers.append(layers.InputLayer(input_shape=(input_dim,)))
            encoder_layers.append(layers.Dense(latent_dim * 2))  # mean and log-variance

        self.encoder = tf.keras.Sequential(encoder_layers)

        decoder_layers = []
        if len(layers_config) > 0:
            decoder_layers.append(layers.InputLayer(input_shape=(latent_dim,)))
            for units in reversed(layers_config):
                decoder_layers.append(layers.Dense(units, activation='relu'))
                decoder_layers.append(layers.BatchNormalization())
            decoder_layers.append(layers.Dense(input_dim, activation='sigmoid'))
        else:
            decoder_layers.append(layers.InputLayer(input_shape=(latent_dim,)))
            decoder_layers.append(layers.Dense(input_dim, activation='sigmoid'))

        self.decoder = tf.keras.Sequential(decoder_layers)

        # Classification head
        self.classifier = tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(n_classes, activation='softmax')  #multiclass classification
        ])

    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(tf.shape(X_train_imputed)[0], latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

    def classify(self, z):
        return self.classifier(z)

# Loss functions
def vae_loss(x, x_decoded_mean, z_mean, z_logvar):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    x_decoded_mean = tf.convert_to_tensor(x_decoded_mean, dtype=tf.float32)
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_decoded_mean, labels=x)
    cross_entropy = tf.reduce_sum(cross_entropy, axis=1)
    kl_divergence = -0.5 * tf.reduce_sum(1 + z_logvar - tf.square(z_mean) - tf.exp(z_logvar), axis=1)
    return tf.reduce_mean(cross_entropy + kl_divergence)

def classification_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred))

# Define encoder and decoder layer configurations
encoder_decoder_layers = [int(input_dim * 0.5), int(input_dim * 0.3)]

vae = VAE(input_dim, latent_dim, encoder_decoder_layers, n_classes)

optimizer = tf.keras.optimizers.Adam()

def train_step(x, y):
    with tf.GradientTape() as tape:
        z_mean, z_logvar = vae.encode(x)
        z = vae.reparameterize(z_mean, z_logvar)
        x_decoded_mean = vae.decode(z)
        class_output = vae.classify(z)
        vae_reconstruction_loss = vae_loss(x, x_decoded_mean, z_mean, z_logvar)
        clf_loss = classification_loss(y, class_output)
        total_loss = vae_reconstruction_loss + clf_loss
    gradients = tape.gradient(total_loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(gradients, vae.trainable_variables))
    return total_loss, vae_reconstruction_loss, clf_loss

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=10, restore_best_weights=True
)


# Training loop with early stopping
epochs = 10000
batch_size = 64
for epoch in range(epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_vae_loss_avg = tf.keras.metrics.Mean()
    epoch_clf_loss_avg = tf.keras.metrics.Mean()

    for i in range(0, len(X_train_imputed), batch_size):
        x_batch = X_train_imputed.iloc[i:i+batch_size].values.astype(np.float32)
        y_batch = y_train[i:i+batch_size].astype(np.int64)
        loss, vae_loss_val, clf_loss_val = train_step(x_batch, y_batch)
        epoch_loss_avg.update_state(loss)
        epoch_vae_loss_avg.update_state(vae_loss_val)
        epoch_clf_loss_avg.update_state(clf_loss_val)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {epoch_loss_avg.result().numpy()}, VAE Loss: {epoch_vae_loss_avg.result().numpy()}, Classification Loss: {epoch_clf_loss_avg.result().numpy()}")

# Impute missing values using VAE
def vae_imputation(data, vae, column_names):
    data = tf.convert_to_tensor(data, dtype=tf.float32)
    z_mean, _ = vae.encode(data)
    imputed_data = vae.sample(z_mean).numpy()
    return pd.DataFrame(imputed_data, columns=column_names)

# Perform VAE imputation on the training set
X_train_imputed = vae_imputation(X_train_imputed, vae, X_train.columns)

# Perform VAE imputation on the testing set
X_test_imputed = vae_imputation(X_test_imputed, vae, X_test.columns)

# Now use X_train_imputed and X_test_imputed for further modeling


Epoch 0, Loss: 63.732093811035156, VAE Loss: 62.7570915222168, Classification Loss: 0.9750049710273743
Epoch 10, Loss: 43.928279876708984, VAE Loss: 43.117618560791016, Classification Loss: 0.8106594681739807
Epoch 20, Loss: 42.61949157714844, VAE Loss: 41.83378601074219, Classification Loss: 0.7857041954994202
Epoch 30, Loss: 42.463985443115234, VAE Loss: 41.71138000488281, Classification Loss: 0.752602219581604
Epoch 40, Loss: 42.340152740478516, VAE Loss: 41.671897888183594, Classification Loss: 0.6682577729225159


In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Hyperparameters
hint_rate = 0.9
alpha = 10
batch_size = 64
epochs = 100
learning_rate = 0.001

# Assuming X_train and X_test are your dataframes with missing values

# Impute missing values with a simple strategy (e.g., median) for initial setup
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Normalize the data
scaler = StandardScaler()
X_train_imputed = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=X_train.columns)
X_test_imputed = pd.DataFrame(scaler.transform(X_test_imputed), columns=X_test.columns)

X_train_imputed = X_train_imputed.astype(np.float32)
X_test_imputed = X_test_imputed.astype(np.float32)

input_dim = X_train_imputed.shape[1]

# Binary mask matrix indicating missing values (1 if missing, 0 if not)
def binary_mask(data):
    return np.where(data.isna(), 1, 0).astype(np.float32)

# Hint matrix
def hint_matrix(data, hint_rate):
    mask = binary_mask(data)
    hint = mask * np.random.binomial(1, hint_rate, data.shape)
    return hint.astype(np.float32)

# Generator model
def build_generator(input_dim):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(input_dim * 2,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(input_dim, activation='sigmoid')
    ])
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(input_dim * 2,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(input_dim, activation='sigmoid')
    ])
    return model

# Define models
generator = build_generator(input_dim)
discriminator = build_discriminator(input_dim)

# Optimizers
gen_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
disc_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Loss function for discriminator
def discriminator_loss(real, fake):
    real_loss = tf.keras.losses.binary_crossentropy(real, tf.ones_like(real))
    fake_loss = tf.keras.losses.binary_crossentropy(fake, tf.zeros_like(fake))
    return real_loss + fake_loss

# Loss function for generator
def generator_loss(fake):
    return tf.keras.losses.binary_crossentropy(fake, tf.ones_like(fake))

# Training step
@tf.function
def train_step(X, M, H):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        G_sample = generator(tf.concat([X, M], axis=1))
        H_concat = tf.concat([G_sample, H], axis=1)
        D_real = discriminator(tf.concat([X, H], axis=1))
        D_fake = discriminator(H_concat)

        gen_loss = generator_loss(D_fake)
        disc_loss = discriminator_loss(D_real, D_fake)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
for epoch in range(epochs):
    for start in range(0, len(X_train_imputed), batch_size):
        end = start + batch_size
        X_batch = X_train_imputed[start:end]
        M_batch = binary_mask(X_train.iloc[start:end])
        H_batch = hint_matrix(X_train.iloc[start:end], hint_rate)

        gen_loss, disc_loss = train_step(X_batch, M_batch, H_batch)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()}")

# Impute missing values using the generator
def impute_missing_values(data, generator, max_iterations=25):
    data_copy = data.copy()
    for _ in range(max_iterations):
        M = binary_mask(data_copy)
        G_sample = generator(tf.concat([data_copy, M], axis=1))
        data_copy[M == 1] = G_sample[M == 1]
    return data_copy

# Perform imputation on the training and testing sets
X_train_imputed = impute_missing_values(X_train_imputed, generator)
X_test_imputed = impute_missing_values(X_test_imputed, generator)

# Scale the imputed data back to the original scale
X_train_imputed = pd.DataFrame(scaler.inverse_transform(X_train_imputed), columns=X_train.columns)
X_test_imputed = pd.DataFrame(scaler.inverse_transform(X_test_imputed), columns=X_test.columns)




Epoch 0, Generator Loss: [8.600904 8.274394 8.288845], Discriminator Loss: [13.762307 11.978455 13.657398]
Epoch 10, Generator Loss: [15.332672 15.297342 15.329958], Discriminator Loss: [0.002743   0.03611084 0.00493603]
Epoch 20, Generator Loss: [15.33322  15.330939 15.327589], Discriminator Loss: [0.00109405 0.00231248 0.0057365 ]
Epoch 30, Generator Loss: [15.333235 15.332812 15.332914], Discriminator Loss: [0.00042152 0.00042777 0.00036   ]
Epoch 40, Generator Loss: [15.333219 15.332997 15.333132], Discriminator Loss: [2.6641859e-05 2.4221801e-04 1.2417193e-04]
Epoch 50, Generator Loss: [15.333237 15.332973 15.333182], Discriminator Loss: [7.9084013e-05 2.6766260e-04 6.4956475e-05]
Epoch 60, Generator Loss: [15.333238 15.333067 15.333213], Discriminator Loss: [6.4812069e-05 1.7227868e-04 4.2206386e-05]
Epoch 70, Generator Loss: [15.333237 15.333114 15.333224], Discriminator Loss: [3.4358000e-06 1.2615701e-04 2.3804281e-05]
Epoch 80, Generator Loss: [15.333238 15.332993 15.333219], 

In [55]:
#Normalization of the data:

min_max_scaler=MinMaxScaler()
X_train_normalized=min_max_scaler.fit_transform(X_train_imputed)
X_test_normalized=min_max_scaler.transform(X_test_imputed)


In [56]:
#Feature selection:

MI=mutual_info_classif(X_train_normalized,y_train)

In [57]:
MI.shape

(60,)

In [58]:
#using MI selecting the top n features:

n_features=15
selected_scores_indices=np.argsort(MI)[::-1][0:n_features]

In [59]:
#top n_features names:

top_n_feature_names=df.columns[selected_scores_indices]
print("Top", n_features,"features:")
print(top_n_feature_names)

Top 15 features:
Index(['NIBAN1', 'OTULINL', 'RNF186', 'CDHR2', 'CRYBG2', 'NEURL1B', 'ANGPTL4',
       'ESRP1', 'NDFIP2', 'DAO', 'SIDT1', 'RNF43', 'SIAE', 'CLIC5', 'SAMD9'],
      dtype='object')


In [60]:
X_train_selected=X_train_normalized[:,selected_scores_indices]
X_test_selected=X_test_normalized[:,selected_scores_indices]

In [37]:
#model building:

RF=OneVsRestClassifier(RandomForestClassifier(max_features=0.2))
RF.fit(X_train_selected,y_train)
y_pred=RF.predict(X_test_selected)
pred_prob=RF.predict_proba(X_test_selected)

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # Model training
    RF = OneVsRestClassifier(RandomForestClassifier(max_features=0.2))
    RF.fit(X_train_selected, y_train)

    # Prediction
    y_pred = RF.predict(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


Classification Report over 50 runs:

Accuracy: Mean = 0.93, Best = 0.94, Std Dev = 0.0052
Precision: Mean = 0.93, Best = 0.94, Std Dev = 0.0046
Recall: Mean = 0.93, Best = 0.94, Std Dev = 0.0052
F1 Score: Mean = 0.93, Best = 0.94, Std Dev = 0.0052


In [65]:
#XGBoost


from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from xgboost import XGBClassifier

# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 100
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # Model training
    XGB = OneVsRestClassifier(XGBClassifier(max_features=0.2))
    XGB.fit(X_train_selected, y_train)
    #prediction
    y_pred = XGB.predict(X_test_selected)
    pred_prob = XGB.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:46:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:46:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:46:30] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:46:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:46:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.



Classification Report over 50 runs:

Accuracy: Mean = 0.91, Best = 0.91, Std Dev = 0.0000
Precision: Mean = 0.92, Best = 0.92, Std Dev = 0.0000
Recall: Mean = 0.91, Best = 0.91, Std Dev = 0.0000
F1 Score: Mean = 0.91, Best = 0.91, Std Dev = 0.0000


In [ ]:
#Gradient Boosting:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 100
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # Model training
    GB = OneVsRestClassifier(GradientBoostingClassifier(max_features=0.2))
    GB.fit(X_train_selected, y_train)
    #prediction:
    y_pred = GB.predict(X_test_selected)
    pred_prob = GB.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#logistic regression:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 100
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
  #model building
    LR = OneVsRestClassifier(LogisticRegression(max_iter=1000))
    LR.fit(X_train_selected, y_train)
    #prediction
    y_pred = LR.predict(X_test_selected)
    pred_prob = LR.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#Adaboost:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.ensemble import AdaBoostClassifier


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # model building:
    AdaBoost = OneVsRestClassifier(AdaBoostClassifier())
    AdaBoost.fit(X_train_selected, y_train)

    #prediction
    y_pred = AdaBoost.predict(X_test_selected)
    pred_prob = AdaBoost.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#MLP:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.neural_network import MLPClassifier


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 100
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # model building:
    MLP = OneVsRestClassifier(MLPClassifier())
    MLP.fit(X_train_selected, y_train)
    y_pred = MLP.predict(X_test_selected)
    pred_prob = MLP.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#svc

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 100
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    from sklearn.svm import SVC

    # model building:
    SVM = OneVsRestClassifier(SVC(probability=True))
    SVM.fit(X_train_selected, y_train)

    #prediction
    y_pred = SVM.predict(X_test_selected)
    pred_prob = SVM.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#KNN

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):

    knn_with_probs = KNeighborsClassifier(n_neighbors=5, weights='distance')
    knn_with_probs.fit(X_train_selected, y_train)
    #prediction
    y_pred= knn_with_probs.predict(X_test_selected)
    pred_prob = knn_with_probs.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [66]:
 #hyperparameter tuning-MLP

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.multiclass import OneVsRestClassifier

    # Define your MLP Classifier
    mlp = MLPClassifier()

    # Create an instance of OneVsRestClassifier
    ovr_mlp = OneVsRestClassifier(mlp)

    # Define a grid of hyperparameters to search
    param_grid = {
      'estimator__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Sizes of the hidden layers
      'estimator__activation': ['tanh', 'relu'],  # Activation function for the hidden layer
      'estimator__solver': ['sgd', 'adam'],  # Solver for weight optimization
      'estimator__alpha': [0.0001, 0.05],  # L2 penalty (regularization term) parameter
      'estimator__learning_rate': ['constant', 'adaptive'],  # Learning rate schedule for weight updates
      # Add other hyperparameters if needed
    }

    # Create a GridSearchCV object
    grid = GridSearchCV(ovr_mlp, param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the data
    grid.fit(X_train_selected, y_train)

    # Best hyperparameters
    print("Best hyperparameters: ", grid.best_params_)

    # Best model
    best_model = grid.best_estimator_

    # Make predictions with the best model
    y_pred = best_model.predict(X_test_selected)
    pred_prob = best_model.predict_proba(X_test_selected)


    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Se

Best hyperparameters:  {'estimator__C': 100, 'estimator__solver': 'newton-cg'}


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Se

Best hyperparameters:  {'estimator__C': 100, 'estimator__solver': 'newton-cg'}


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Se

...

Best hyperparameters:  {'estimator__C': 100, 'estimator__solver': 'newton-cg'}


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Se

Best hyperparameters:  {'estimator__C': 100, 'estimator__solver': 'newton-cg'}


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Se

Best hyperparameters:  {'estimator__C': 100, 'estimator__solver': 'newton-cg'}
Classification Report over 50 runs:

Accuracy: Mean = 0.93, Best = 0.93, Std Dev = 0.0000
Precision: Mean = 0.93, Best = 0.93, Std Dev = 0.0000
Recall: Mean = 0.93, Best = 0.93, Std Dev = 0.0000
F1 Score: Mean = 0.93, Best = 0.93, Std Dev = 0.0000


In [38]:
#model evaluation:

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
report=classification_report(y_test,y_pred,target_names=labels)
print('classification report\n\n')
print(report)
print('Accuracy_score',accuracy)

classification report


              precision    recall  f1-score   support

      normal       1.00      1.00      1.00        80
     tumoral       1.00      1.00      1.00        81

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161

Accuracy_score 1.0


In [ ]:
#hyperparameter tuning-RandomForest

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    from sklearn.model_selection import GridSearchCV
    from sklearn.multiclass import OneVsRestClassifier
    # Define your Random Forest Classifier
    rf = RandomForestClassifier()

    # Create an instance of OneVsRestClassifier
    ovr_rf = OneVsRestClassifier(rf)

    # Define a grid of hyperparameters to search
    param_grid = {
    'estimator__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'estimator__max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'estimator__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'estimator__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
     'estimator__max_features': [0.1, 0.2, 0.3],  # Number of features to consider when looking for the best split

    }

    # Create a GridSearchCV object
    grid = GridSearchCV(ovr_rf, param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the data
    grid.fit(X_train_selected, y_train)

    # Best hyperparameters
    print("Best hyperparameters: ", grid.best_params_)

    # Best model
    best_model = grid.best_estimator_

    # Make predictions with the best model
    y_pred = best_model.predict(X_test_selected)
    pred_prob = best_model.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-Adaboost

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # Define your AdaBoost Classifier
    ada_boost = AdaBoostClassifier()

    # Create an instance of OneVsRestClassifier
    ovr_ada = OneVsRestClassifier(ada_boost)

    # Define a grid of hyperparameters to search
    param_grid = {
        'estimator__n_estimators': [50, 100, 200],  # Number of weak learners to train iteratively
        'estimator__learning_rate': [0.01, 0.1, 1],  # It contributes to the weights of weak learners
       # Add other hyperparameters if needed
    }

    # Create a GridSearchCV object
    grid = GridSearchCV(ovr_ada, param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the data
    grid.fit(X_train_selected, y_train)

    # Best hyperparameters
    print("Best hyperparameters: ", grid.best_params_)

    # Best model
    best_model = grid.best_estimator_

    # Make predictions with the best model
    y_pred = best_model.predict(X_test_selected)
    pred_prob = best_model.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)


# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-GradientBoost

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.multiclass import OneVsRestClassifier

# # Define your Gradient Boosting Classifier
# gbc = GradientBoostingClassifier()

# # Create an instance of OneVsRestClassifier
# ovr_gbc = OneVsRestClassifier(gbc)

# # Define a grid of hyperparameters to search
# param_grid = {
#     'estimator__n_estimators': [100, 200, 300],  # Number of boosting stages to be run
#     'estimator__learning_rate': [0.01, 0.1, 0.2],  # Learning rate shrinks the contribution of each tree
#     'estimator__max_depth': [3, 4, 5],  # Maximum depth of the individual regression estimators
#     'estimator__max_features': [0.1, 0.2, 0.3],  # Number of features to consider when looking for the best split
#     # Add other hyperparameters if needed
# }

# # Create a GridSearchCV object
# grid = GridSearchCV(ovr_gbc, param_grid, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid.fit(X_train_selected, y_train)

# # Best hyperparameters
# print("Best hyperparameters: ", grid.best_params_)

# # Best model
# best_model = grid.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_test_selected)
# pred_prob = best_model.predict_proba(X_test_selected)



    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-XGB

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

# Define your XGBoost Classifier
xgb = XGBClassifier()

# Create an instance of OneVsRestClassifier
ovr_xgb = OneVsRestClassifier(xgb)

# Define a grid of hyperparameters to search
param_grid = {
    'estimator__n_estimators': [100, 200, 300],  # Number of gradient boosted trees
    'estimator__learning_rate': [0.01, 0.1, 0.2],  # Boosting learning rate
    'estimator__max_depth': [3, 4, 5],  # Maximum tree depth for base learners
    'estimator__subsample': [0.7, 0.8, 0.9],  # Subsample ratio of the training instances
    'estimator__colsample_bytree': [0.7, 0.8, 0.9],  # Subsample ratio of columns when constructing each tree
    # Add other hyperparameters if needed
}

# Create a GridSearchCV object
grid = GridSearchCV(ovr_xgb, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid.fit(X_train_selected, y_train)

# Best hyperparameters
print("Best hyperparameters: ", grid.best_params_)

# Best model
best_model = grid.best_estimator_

# Make predictions with the best model
y_pred = best_model.predict(X_test_selected)
pred_prob = best_model.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-SVC

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    #hyperparameter tuning-SVC

# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# from sklearn.multiclass import OneVsRestClassifier

# # Define your Support Vector Classifier
# svc = SVC(probability=True)

# # Create an instance of OneVsRestClassifier
# ovr_svc = OneVsRestClassifier(svc)

# # Define a grid of hyperparameters to search
# param_grid = {
#     'estimator__C': [0.1, 1, 10],  # Regularization parameter
#     'estimator__kernel': ['linear', 'rbf', 'poly'],  # Specifies the kernel type to be used in the algorithm
#     'estimator__gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
#     # Add other hyperparameters if needed
# }

# # Create a GridSearchCV object
# grid = GridSearchCV(ovr_svc, param_grid, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid.fit(X_train_selected, y_train)

# # Best hyperparameters
# print("Best hyperparameters: ", grid.best_params_)

# # Best model
# best_model = grid.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_test_selected)
# pred_prob = best_model.predict_proba(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-KNN

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
    # Define your KNeighbors Classifier with distance-based weights
    knn = KNeighborsClassifier(weights='distance')

    # Define a grid of hyperparameters to search
    param_grid = {
      'n_neighbors': [3, 5, 7, 10],  # Number of neighbors
      'metric': ['euclidean', 'manhattan', 'minkowski','cosine'],  # Distance metric

       }

# # Create a GridSearchCV object
# grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid.fit(X_train_selected, y_train)

# # Best hyperparameters
# print("Best hyperparameters: ", grid.best_params_)

# # Best model
# best_model = grid.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_test_selected)
# pred_prob = best_model.predict_proba(X_test_selected)


    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')


In [ ]:
#hyperparameter tuning-MLP

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Assuming X_train_selected, y_train, X_test_selected, and y_test are defined

num_runs = 50
accuracies = []
precisions = []
recalls = []
f1_scores = []

for _ in range(num_runs):
   # from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.multiclass import OneVsRestClassifier

# # Define your MLP Classifier
# mlp = MLPClassifier()

# # Create an instance of OneVsRestClassifier
# ovr_mlp = OneVsRestClassifier(mlp)

# # Define a grid of hyperparameters to search
# param_grid = {
#     'estimator__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Sizes of the hidden layers
#     'estimator__activation': ['tanh', 'relu'],  # Activation function for the hidden layer
#     'estimator__solver': ['sgd', 'adam'],  # Solver for weight optimization
#     'estimator__alpha': [0.0001, 0.05],  # L2 penalty (regularization term) parameter
#     'estimator__learning_rate': ['constant', 'adaptive'],  # Learning rate schedule for weight updates
#     # Add other hyperparameters if needed
# }

# # Create a GridSearchCV object
# grid = GridSearchCV(ovr_mlp, param_grid, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid.fit(X_train_selected, y_train)

# # Best hyperparameters
# print("Best hyperparameters: ", grid.best_params_)

# # Best model
# best_model = grid.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_test_selected)
# pred_prob = best_model.predict_proba(X_test_selected)


    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # Store metrics
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate mean, best performance, and standard deviation
mean_accuracy = np.mean(accuracies)
best_accuracy = np.max(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
best_precision = np.max(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
best_recall = np.max(recalls)
std_recall = np.std(recalls)

mean_f1 = np.mean(f1_scores)
best_f1 = np.max(f1_scores)
std_f1 = np.std(f1_scores)

# Print the results
print('Classification Report over 50 runs:\n')
print(f'Accuracy: Mean = {mean_accuracy:.2f}, Best = {best_accuracy:.2f}, Std Dev = {std_accuracy:.4f}')
print(f'Precision: Mean = {mean_precision:.2f}, Best = {best_precision:.2f}, Std Dev = {std_precision:.4f}')
print(f'Recall: Mean = {mean_recall:.2f}, Best = {best_recall:.2f}, Std Dev = {std_recall:.4f}')
print(f'F1 Score: Mean = {mean_f1:.2f}, Best = {best_f1:.2f}, Std Dev = {std_f1:.4f}')
